In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## 1python中的embedding

In [50]:
word2idx={'hello':0,'world':1} 
embeds=nn.Embedding(2,5)
#lookup_tensor=torch.Tensor([word2idx['hello']],dtype=torch.LongTensor)
lookup_tensor = torch.LongTensor([word2idx["hello"]])
# 新建立的tensor一定是torch.tensor([]),[]不要忘了
hello_embed=embeds(lookup_tensor)
# hello_embed的输入，一定是一个int，一般都用longtensor
inputs=torch.LongTensor([0,1])
embed=embeds(inputs).view(1,-1)  #经过view变换后，仍然是一个矩阵，1*n，如果要变成向量，还需要flatten
embed=embed.flatten()
print(embed)





tensor([-0.8328,  0.4381, -0.9808, -0.7558,  0.7410, -0.1233,  0.6379,  0.9327,
         0.9399,  2.2820], grad_fn=<AsStridedBackward>)


## 2 language model

In [62]:
# 1装备数据和word2idx
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

EMBEDDING_DIM=10
CONTEXT_SIZE=2
VOCAB_SIZE=len(vocab)

trigrams=[([test_sentence[i-1],test_sentence[i]],test_sentence[i+1]) for i in range(1,len(test_sentence)-1)]
vocab=list(set(test_sentence))
word2idx={word:i for i,word in enumerate(vocab)}
#for context,target in trigrams:
    #print('context:{},target:{}'.format(context,target))







In [133]:
# 2建立模型
class nGramModel(nn.Module):
    def __init__(self,embedding_dim,context_size,vocab_size):
        super(nGramModel,self).__init__()
        self.embeddings=nn.Embedding(vocab_size,embedding_dim)
        self.linear1=nn.Linear(context_size*embedding_dim,128)
        self.linear2=nn.Linear(128,vocab_size)
    def forward(self,inputs):
        embeds=self.embeddings(inputs).view(1,-1)
        out=self.linear1(embeds)
        out=F.relu(out)
        out=self.linear2(out)
        #print(out)
        log_prob=F.log_softmax(out,dim=1)
        #注意，这里dim描述的是out的形式，如果dim输入不正确，log_prob可能计算为0矩阵。
        #print(log_prob)
        return log_prob
model=nGramModel(EMBEDDING_DIM,CONTEXT_SIZE,VOCAB_SIZE)
    
        
        
        

## 3 关于model的训练
1 model=nGramModel(EMBEDDING_DIM,CONTEXT_SIZE,VOCAB_SIZE)这句话，实际上是在执行构造函数

2执行完构造函数后，model就成为了forward的引用，之后model就是forward函数。所以要计算前向结果，直接model（input）

3nn.NLLLoss()的输入很奇怪，同时需要总结一下其他非线性函数

In [135]:

def train(model):
    loss_function = nn.NLLLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    for epoch in range(100):
        loss_list=[]
        model.zero_grad()
        for context,label in trigrams:
            inputs=torch.LongTensor([word2idx[context[0]],word2idx[context[1]]])
            target=torch.LongTensor([word2idx[label]])
            #这里，还需要仔细了解nn.NLLLoss，这个loss函数的输入时一个tensor向量，一个标量，很奇怪。
            log_prob=model(inputs)
            loss=loss_function(log_prob,target)
            loss.backward()
            optimizer.step()
            loss_list.append(loss)
        loss_total=sum(loss_list)
        print('Epoch{}: loss={}'.format(epoch,loss_total))

        
train(model)  



         
            
            
            
        

Epoch0: loss=50.0132942199707
Epoch1: loss=46.42011642456055
Epoch2: loss=40.404701232910156
Epoch3: loss=38.716468811035156
Epoch4: loss=34.52384948730469
Epoch5: loss=33.47443389892578
Epoch6: loss=30.295167922973633
Epoch7: loss=29.472652435302734
Epoch8: loss=26.973745346069336
Epoch9: loss=26.282020568847656
Epoch10: loss=24.347497940063477
Epoch11: loss=23.791004180908203
Epoch12: loss=22.35445785522461
Epoch13: loss=21.885984420776367
Epoch14: loss=20.841094970703125
Epoch15: loss=20.42546844482422
Epoch16: loss=19.641998291015625
Epoch17: loss=19.22689437866211
Epoch18: loss=18.605792999267578
Epoch19: loss=18.18592071533203
Epoch20: loss=17.682117462158203
Epoch21: loss=17.249794006347656
Epoch22: loss=16.7958984375
Epoch23: loss=16.381576538085938
Epoch24: loss=15.954948425292969
Epoch25: loss=15.55398178100586
Epoch26: loss=15.157174110412598
Epoch27: loss=14.772966384887695
Epoch28: loss=14.396240234375
Epoch29: loss=14.030515670776367
Epoch30: loss=13.675812721252441
Epoch

In [140]:
loss_function = nn.NLLLoss()
loss_list=[]
for context,label in trigrams:
    inputs=torch.LongTensor([word2idx[context[0]],word2idx[context[1]]])
    target=torch.LongTensor([word2idx[label]])       
    log_prob=model(inputs)
    loss=loss_function(log_prob,target)
    loss_list.append(loss)
loss_total=sum(loss_list)
print("Test result: loss=",loss_total)
#再一次证明，在training函数中训练model后，model的参数却是改变了，不用担心model是可变变量还是不可变变量

Test result: loss= tensor(4.1054, grad_fn=<AddBackward0>)
